<a href="https://colab.research.google.com/github/nlquantumm-source/RT-Control-Readout-Electronics/blob/main/RT_Control_%26_Readout_Electronics_MuMax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Install and Compile MuMax3 (Optimized Mock Compiler Strategy).

This script detects the GPU, manually compiles CUDA kernels,
and mocks the compiler. It includes checks to skip re-compilation
if MuMax3 is already installed and functional.
"""

import glob
import os
import shutil
import subprocess
import sys

# Set working directory
os.chdir("/content")

print("=== 1. Setup Environment ===")

# Optimization: Check if MuMax3 is already installed to avoid redundant work
FORCE_RECOMPILE = False
if shutil.which("mumax3") and not FORCE_RECOMPILE:
    try:
        # Verify it actually runs
        subprocess.run(
            "mumax3 -v",
            shell=True,
            check=True,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
        print("MuMax3 is already installed and functional. Skipping compilation.")
        print("Set FORCE_RECOMPILE = True in the script to force a rebuild.")
        # Exit the script early to save time
        # In a notebook, sys.exit() stops this cell but not the kernel
    except subprocess.CalledProcessError:
        print("MuMax3 found but non-functional. Proceeding with re-installation.")
else:
    print("MuMax3 not found or force recompile requested. Starting build...")

# Proceed with installation if not skipped
if not shutil.which("mumax3") or FORCE_RECOMPILE:
    # Install dependencies
    subprocess.run(
        "apt update -qq && apt install -qq git build-essential wget",
        shell=True,
        check=True
    )

    # Install Go 1.21 (Stable)
    if not os.path.exists("/usr/local/go/bin/go"):
        print("Installing Go...")
        commands = [
            "wget -q https://go.dev/dl/go1.21.6.linux-amd64.tar.gz",
            "tar -C /usr/local -xzf go1.21.6.linux-amd64.tar.gz",
            "rm go1.21.6.linux-amd64.tar.gz"
        ]
        subprocess.run(" && ".join(commands), shell=True, check=True)

    # Set paths
    os.environ["PATH"] += ":/usr/local/go/bin:/root/go/bin"
    os.environ["GOPATH"] = "/root/go"

    # Check GPU and determine Architecture
    print("\n=== 2. Detect GPU Architecture ===")
    try:
        gpu_info = subprocess.check_output(
            "nvidia-smi --query-gpu=name --format=csv,noheader", shell=True
        ).decode().strip()
        print(f"Detected GPU: {gpu_info}")

        # Architecture Target
        # L4/Ada/A100 -> sm_80 (Ampere) + PTX
        if "L4" in gpu_info or "Ada" in gpu_info or "A100" in gpu_info:
            arch_flag = "-gencode arch=compute_80,code=compute_80"
        elif "V100" in gpu_info:
            arch_flag = "-gencode arch=compute_70,code=compute_70"
        else:
            # Fallback to Turing (T4)
            arch_flag = "-gencode arch=compute_75,code=compute_75"

        print(f"Selected Compilation Flags: {arch_flag}")
    except subprocess.CalledProcessError:
        print("WARNING: Could not detect GPU. Defaulting to sm_75.")
        arch_flag = "-gencode arch=compute_75,code=compute_75"

    # Clone MuMax3
    print("\n=== 3. Clone MuMax3 ===")
    if os.path.exists("3"):
        shutil.rmtree("3")

    print("Cloning mumax3 repository (Stable v3.10)...")
    subprocess.run(
        ["git", "clone", "--branch", "v3.10", "https://github.com/mumax/3", "3"],
        check=True
    )

    # Patch: CUDA 12 Compatibility
    DUMMY_MODE_GO = """
    package cufft
    type CompatibilityMode int
    const (
    	CompatNative CompatibilityMode = 0
    	CompatFFTWPadding CompatibilityMode = 1
    	CompatFFTWAsymmetric CompatibilityMode = 2
    	CompatFFTWAll CompatibilityMode = 3
    )
    """
    with open("3/cuda/cufft/mode.go", "w", encoding="utf-8") as f:
        f.write(DUMMY_MODE_GO)

    # Manual Compilation of Kernels
    print("\n=== 4. Compile CUDA Kernels (Manual) ===")
    cuda_dir = os.path.abspath("3/cuda")

    # 4.1. Run REAL NVCC to generate PTX files
    print("Compiling .cu files to .ptx...")
    try:
        cu_files = glob.glob(os.path.join(cuda_dir, "*.cu"))
        nvcc_cmd = ["nvcc", "-ptx"] + arch_flag.split() + cu_files
        subprocess.run(
            nvcc_cmd,
            cwd=cuda_dir,
            check=True,
            capture_output=True
        )
        print(f"Successfully compiled {len(cu_files)} kernels.")
    except subprocess.CalledProcessError as e:
        print("CRITICAL: NVCC Compilation failed.")
        print(e.stderr.decode())
        sys.exit(1)

    # 4.2. Setup Mock NVCC
    mock_bin_dir = os.path.abspath("mock_bin")
    os.makedirs(mock_bin_dir, exist_ok=True)
    mock_nvcc_path = os.path.join(mock_bin_dir, "nvcc")

    with open(mock_nvcc_path, "w", encoding="utf-8") as f:
        f.write("#!/bin/sh\n")
        f.write("echo 'Mock NVCC: Doing nothing (success)'\n")
        f.write("exit 0\n")
    subprocess.run(f"chmod +x {mock_nvcc_path}", shell=True, check=True)

    # Prepend mock bin to PATH
    os.environ["PATH"] = f"{mock_bin_dir}:" + os.environ["PATH"]
    print("Mock NVCC installed in PATH.")

    # 4.3. Run cuda2go to Package kernels
    print("Packaging kernels using cuda2go...")
    print("Initializing Go module...")
    subprocess.run(
        "go mod init github.com/mumax/3",
        cwd="/content/3",
        shell=True,
        check=True
    )
    subprocess.run(
        "go mod tidy",
        cwd="/content/3",
        shell=True,
        check=True
    )

    try:
        subprocess.run(
            "go run cuda2go.go",
            cwd=cuda_dir,
            shell=True,
            check=True
        )
        print("Go wrapper (fatbin.go) generated successfully.")
    except subprocess.CalledProcessError:
        print("CRITICAL: Failed to run cuda2go.go")
        sys.exit(1)

    # Install Main Binary
    print("\n=== 5. Install MuMax3 Binary ===")
    subprocess.run("go clean -cache", shell=True, check=True)
    try:
        subprocess.run(
            "go install -v -ldflags '-s -w' ./cmd/mumax3",
            cwd="/content/3",
            shell=True,
            check=True,
            env=dict(os.environ, CGO_ENABLED="1"),
            capture_output=True,
            text=True
        )
        print("MuMax3 installed successfully.")
    except subprocess.CalledProcessError as e:
        print("CRITICAL: Go Install failed.")
        print("--- STDERR ---")
        print(e.stderr)
        print("--- STDOUT ---")
        print(e.stdout)
        sys.exit(1)

    # Move to path
    subprocess.run(
        "cp /root/go/bin/mumax3 /usr/local/bin/mumax3",
        shell=True,
        check=True
    )
    subprocess.run("chmod +x /usr/local/bin/mumax3", shell=True, check=True)

# Verify
print("\n=== 6. Verification ===")
subprocess.run("mumax3 -v", shell=True, check=False)
print("\nDone! Please run Step 5 (or 4) again.")

## Simulation Parameters & Reference Summary

### 1. QPSK Signal Parameters (Input $x(t)$)
*   **Local Oscillator ($\omega_{LO}$):** $2\pi \times 5$ GHz ($10\pi$ Grad/s) [Adjustable 1-10 GHz]
*   **Amplitudes ($I, Q$):** 1.0 (Normalized) [Adjustable 0.1-5.0]
*   **Timebase:** $t_{max} = 1$ ns, $dt = 1$ ps
*   **Equation:** $x(t) = I \cdot \cos(\omega_{LO} t) - Q \cdot \sin(\omega_{LO} t)$

### 2. Nonlinear System Coefficients (Output $y(t)$)
Used to model RF Control/Readout Electronics nonlinearity.
*   **Model:** $y(t) = \alpha_1 x + \alpha_2 x^2 + \alpha_3 x^3$
*   **$\alpha_1$ (Linear Gain):** 1.0
*   **$\alpha_2$ (Quadratic/IMD2):** 0.1
*   **$\alpha_3$ (Cubic/IMD3):** 0.01
*   **Derived Metrics:** 1dB Compression Point, IP3 Point.

### 3. Micromagnetic Simulation (MuMax3)
*   **Material:** Permalloy-like (NiFe)
    *   $M_{sat}$: $800 \times 10^3$ A/m
    *   $A_{ex}$: $13 \times 10^{-12}$ J/m
    *   Damping ($\alpha$): 0.01
*   **Geometry:** $100 \text{ nm} \times 100 \text{ nm} \times 10 \text{ nm}$ (Grid: $50 \times 50 \times 5$)
*   **Excitation:** External Field $B_{ext}$ driven by $y(t)$ along the X-axis.
*   **Software:** MuMax3 v3.10 (Custom compiled with CUDA 12 support).

In [ ]:
"""
Define Equations with Interactive Inputs.

Use sliders for parameters in the equations. Includes optimized
signal generation functions and reusable plotting logic.
"""

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from ipywidgets import interact

# Parameters for 2.3: QPSK Modulation
omega_lo_slider = widgets.FloatSlider(
    value=2 * np.pi * 5e9,
    min=1e9,
    max=10e9,
    step=1e8,
    description='ω_LO (rad/s)'
)
i_amp_slider = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=5.0,
    description='I(t) Amp'
)
q_amp_slider = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=5.0,
    description='Q(t) Amp'
)
t_max_slider = widgets.FloatSlider(
    value=1e-9,
    min=1e-10,
    max=10e-9,
    step=1e-10,
    description='t_max (s)'
)
dt_slider = widgets.FloatSlider(
    value=1e-12,
    min=1e-13,
    max=1e-11,
    description='dt (s)'
)

# Parameters for 2.4: Nonlinear Coefficients
alpha1_slider = widgets.FloatSlider(
    value=1.0,
    min=0.5,
    max=2.0,
    description='α₁'
)
alpha2_slider = widgets.FloatSlider(
    value=0.1,
    min=0,
    max=0.5,
    description='α₂'
)
alpha3_slider = widgets.FloatSlider(
    value=0.01,
    min=0,
    max=0.1,
    description='α₃'
)

# Display widgets
display(
    omega_lo_slider,
    i_amp_slider,
    q_amp_slider,
    t_max_slider,
    dt_slider
)
display(alpha1_slider, alpha2_slider, alpha3_slider)


def generate_xt(omega_lo, i_amp, q_amp, t_max, dt):
    """
    Generate x(t) from 2.3 (simple constant I/Q for demo).

    Args:
        omega_lo (float): Local oscillator frequency in rad/s.
        i_amp (float): Amplitude of In-phase component.
        q_amp (float): Amplitude of Quadrature component.
        t_max (float): Maximum time in seconds.
        dt (float): Time step in seconds.

    Returns:
        tuple: Time array (t) and signal array (x_t).
    """
    t = np.arange(0, t_max, dt)
    # Optimization: Use scalar constants directly (broadcasting)
    # instead of allocating full arrays like np.full_like
    x_t = i_amp * np.cos(omega_lo * t) - q_amp * np.sin(omega_lo * t)
    return t, x_t


def apply_nonlinearity(x_t, alpha1, alpha2, alpha3):
    """
    Compute y(t) from 2.4 nonlinearity.

    Args:
        x_t (numpy.ndarray): Input signal.
        alpha1 (float): Linear coefficient.
        alpha2 (float): Quadratic coefficient.
        alpha3 (float): Cubic coefficient.

    Returns:
        numpy.ndarray: Output signal y(t).
    """
    y_t = alpha1 * x_t + alpha2 * x_t**2 + alpha3 * x_t**3
    return y_t


def compute_metrics(alpha1, alpha3):
    """
    Compute 1 dB compression point and IP3.

    Args:
        alpha1 (float): Linear coefficient.
        alpha3 (float): Cubic coefficient.

    Returns:
        tuple: (A_1dB, A_IP3)
    """
    if alpha3 != 0:
        a_1db = np.sqrt(0.145 * np.abs(alpha1 / alpha3))
        a_ip3 = np.sqrt((4 / 3) * np.abs(alpha1 / alpha3))
    else:
        a_1db = np.inf
        a_ip3 = np.inf
    return a_1db, a_ip3


def create_signal_figure(t, x_t, y_t, a_1db, a_ip3):
    """
    Create matplotlib figure for signals.

    Separated function to allow reuse in download step without duplication.
    """
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

    ax1.plot(t * 1e9, x_t, label='x(t) [Ideal QPSK]')
    ax1.set_xlabel('Time (ns)')
    ax1.set_ylabel('Amplitude')
    ax1.legend()
    ax1.grid(True)

    ax2.plot(t * 1e9, y_t, label='y(t) [Nonlinear]', color='r')
    ax2.set_xlabel('Time (ns)')
    ax2.set_ylabel('Amplitude')
    ax2.legend()
    ax2.grid(True)

    plt.suptitle(f'RF Signals: 1dB Point={a_1db:.2f}, IP3={a_ip3:.2f}')
    plt.tight_layout()
    return fig


@interact
def plot_signals(
    omega_lo=omega_lo_slider,
    i_amp=i_amp_slider,
    q_amp=q_amp_slider,
    t_max=t_max_slider,
    dt=dt_slider,
    alpha1=alpha1_slider,
    alpha2=alpha2_slider,
    alpha3=alpha3_slider
):
    """
    Interactive plot of signals.

    Updates plots based on slider values.
    """
    t, x_t = generate_xt(omega_lo, i_amp, q_amp, t_max, dt)
    y_t = apply_nonlinearity(x_t, alpha1, alpha2, alpha3)
    a_1db, a_ip3 = compute_metrics(alpha1, alpha3)

    fig = create_signal_figure(t, x_t, y_t, a_1db, a_ip3)
    plt.show()

In [ ]:
"""
Prepare MuMax3 Input.

Create a .mx3 script for a ferromagnetic cube driven by Hx(t) = y(t).
We use .mx3 extension to ensure the interpreter handles it correctly.
"""

# MuMax3 input script template
# Updated to use // for comments (Go syntax) instead of #
# Updated TimeStep -> MaxDt
MUMAX_TEMPLATE = """
// Geometry and material
SetGridSize(50, 50, 5)
SetCellSize(2e-9, 2e-9, 2e-9)

// Permalloy-like parameters
Msat = 800e3
Alpha = 0.01
Aex = 13e-12

// Initial state (Uniform magnetization along Z)
m = Uniform(0, 0, 1)

// Define time-dependent drive parameters
// We inject calculated constants directly
w_LO  := {omega}
I_amp := {I}
Q_amp := {Q}
a1    := {a1}
a2    := {a2}
a3    := {a3}
scale := {scale}  // Conversion to Tesla

// Define signals as functions of time (t)
// x(t) = I*cos(w*t) - Q*sin(w*t)
xt := I_amp * cos(w_LO * t) - Q_amp * sin(w_LO * t)

// y(t) = a1*x + a2*x^2 + a3*x^3
yt := a1 * xt + a2 * xt*xt + a3 * xt*xt*xt

// Apply as external magnetic field B_ext (Tesla)
// Drive along X-axis
B_ext = vector(yt * scale, 0, 0)

// Time stepping configuration
t = 0
MaxDt = {dt}  // Limit maximum time step for RF resolution
AutoSave(m, {save_period})  // Save magnetization at fixed interval

// Run simulation
Run({t_max})
Save(m)  // Ensure final state is saved
"""


def write_mumax_script(t_max, dt):
    """
    Write the script with current slider values.

    Args:
        t_max (float): Maximum simulation time.
        dt (float): Time step.
    """
    # 1. Get values from interactive sliders
    # Access global slider widgets directly
    omega = omega_lo_slider.value
    i_val = i_amp_slider.value
    q_val = q_amp_slider.value
    a1 = alpha1_slider.value
    a2 = alpha2_slider.value
    a3 = alpha3_slider.value

    # 2. Scale factor: slider amp -> Tesla
    h_scale = 0.001

    # 3. Pre-calculate save period to avoid script math issues
    # Save 20 frames total or at least every 20 steps
    save_period = dt * 20

    # 4. Format the script
    script = MUMAX_TEMPLATE.format(
        omega=omega,
        I=i_val,
        Q=q_val,
        a1=a1,
        a2=a2,
        a3=a3,
        scale=h_scale,
        t_max=t_max,
        dt=dt,
        save_period=save_period
    )

    # 5. Write to file (Use .mx3 extension)
    with open('simulation.mx3', 'w', encoding='utf-8') as f:
        f.write(script)
    print("Generated 'simulation.mx3' with analytical drive.")

In [ ]:
"""
Run MuMax3 Simulation.

Run the simulation automatically using the generated script.
"""

import os
import shutil
import subprocess
import ipywidgets as widgets
from IPython.display import display


def run_mumax(t_max, dt):
    """
    Generate the script and execute MuMax3.

    Args:
        t_max (float): Simulation duration.
        dt (float): Time step.
    """
    # 1. Generate the script
    write_mumax_script(t_max, dt)

    # Verify the script content (Debugging check)
    with open('simulation.mx3', 'r', encoding='utf-8') as f:
        content = f.read()
        if "MaxDt" in content:
            print("Script syntax check: 'MaxDt' parameter found (Correct).")
        else:
            print("WARNING: 'MaxDt' missing. Please re-run Step 3.")

    print("Checking MuMax3 health... ")
    try:
        # Sanity check
        sanity_script = (
            "SetGridSize(4,4,1); SetCellSize(1e-9,1e-9,1e-9); "
            "m=Uniform(1,0,0); Run(1e-15)"
        )
        subprocess.run(
            ['mumax3', '-s'],
            input=sanity_script,
            check=True,
            text=True,
            capture_output=True
        )
        print("Health check passed. Starting main simulation...")
    except subprocess.CalledProcessError as e:
        print("\nCRITICAL ERROR: MuMax3 failed to run on this GPU.")
        print("Diagnostics (STDERR):\n", e.stderr)
        return

    # 2. Run main simulation
    print("Running Simulation... (Please wait)")
    try:
        # Remove old output to avoid confusion
        if os.path.exists('simulation.out'):
            shutil.rmtree('simulation.out')

        subprocess.run(
            ['mumax3', 'simulation.mx3'],
            check=True,
            capture_output=True,
            text=True
        )
        print("Simulation completed successfully!")
        print("Output data saved to 'simulation.out'. Run Step 5 to visualize.")
    except subprocess.CalledProcessError as e:
        print(f"\nERROR: Simulation failed with return code {e.returncode}.")
        print("--- STDERR ---")
        print(e.stderr)
        print("--- STDOUT ---")
        print(e.stdout)

# Interactive run button
run_button = widgets.Button(description="Run MuMax3", button_style='success')


def on_run_clicked(b):
    """Handle button click event."""
    run_button.disabled = True
    try:
        run_mumax(t_max_slider.value, dt_slider.value)
    finally:
        run_button.disabled = False


run_button.on_click(on_run_clicked)
display(run_button)

# AUTO-RUN: Trigger immediately for user convenience
print("Auto-starting simulation with fixed script...")
run_mumax(t_max_slider.value, dt_slider.value)

In [ ]:
"""
Visualize 3D Simulation Video (MuMax3 Output).

This block loads the output files (.ovf), pre-loads data into memory
for efficiency, and creates a 3D animation.
"""

import glob
import os
import shutil
import subprocess
import sys

import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib import animation

# Install and import discretisedfield (Ubermag) if missing
try:
    import discretisedfield as df
except ImportError:
    print("Installing required library: discretisedfield...")
    subprocess.check_call(
        [sys.executable, "-m", "pip", "install", "-q", "discretisedfield"]
    )
    import discretisedfield as df

# 0. Check for Output and Auto-Run Simulation if missing
output_dir = "simulation.out"
script_file = "simulation.mx3"

# Check if output directory exists and has .ovf files
has_output = (
    os.path.exists(output_dir) and
    len(glob.glob(os.path.join(output_dir, "*.ovf"))) > 0
)

if not has_output:
    print(f"WARNING: Output data missing in '{output_dir}'.")

    if os.path.exists(script_file):
        print(f"Found '{script_file}'. Attempting to run simulation...")

        if os.path.exists(output_dir):
            shutil.rmtree(output_dir)

        try:
            print(f"Executing: mumax3 {script_file}")
            subprocess.run(
                ["mumax3", script_file],
                check=True,
                capture_output=True,
                text=True
            )
            print("Simulation completed successfully! Proceeding to visualization.")
        except subprocess.CalledProcessError as e:
            print("\nERROR: Automatic simulation failed!")
            print("--- MuMax3 STDERR ---")
            print(e.stderr)
            print("--- MuMax3 STDOUT ---")
            print(e.stdout)
            raise RuntimeError(
                "MuMax3 Simulation Failed. See STDERR above for details."
            ) from e
        except FileNotFoundError as e:
            print("ERROR: 'mumax3' command not found. Ensure Step 1 ran successfully.")
            raise e
    else:
        print(f"ERROR: Simulation script '{script_file}' not found.")
        print("Please RUN STEP 3 to generate the input script first.")
        raise FileNotFoundError(f"{script_file} missing")

# 1. Search for Output Files
ovf_files = sorted(glob.glob(os.path.join(output_dir, "*.ovf")))

if not ovf_files:
    print(f"WARNING: No .ovf output files found in '{output_dir}'.")
    print("Cannot generate video. Simulation might have crashed.")
else:
    print(f"Found {len(ovf_files)} frames.")

    # 2. Pre-load Data (Optimization)
    # Reading files from disk during animation is slow.
    # We load them into RAM once.
    print("Pre-loading simulation data into memory... (This speeds up rendering)")
    data_frames = []
    try:
        for fname in ovf_files:
            # Load field
            field = df.Field.from_file(fname)
            # Store the array data only to save memory overhead of full objects
            data_frames.append(field.array)
        print("Data pre-loaded successfully.")

        # 3. Setup Visualization Geometry
        # Load one field to get geometry (static for all frames)
        field0 = df.Field.from_file(ovf_files[0])
        pmin = field0.mesh.region.pmin
        pmax = field0.mesh.region.pmax

        # Optimization: Calculate grid coordinates once
        stride = 4
        all_coords = field0.mesh.coordinate_field().array

        x_coords = all_coords[::stride, ::stride, :, 0].flatten()
        y_coords = all_coords[::stride, ::stride, :, 1].flatten()
        z_coords = all_coords[::stride, ::stride, :, 2].flatten()

        # Create Figure
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')

        def update_frame(frame_idx):
            """Update the 3D plot for the given frame index."""
            ax.clear()

            # Retrieve pre-loaded data
            m_field = data_frames[frame_idx]

            # Extract subsampled vectors
            u_comp = m_field[::stride, ::stride, :, 0].flatten()
            v_comp = m_field[::stride, ::stride, :, 1].flatten()
            w_comp = m_field[::stride, ::stride, :, 2].flatten()

            # Plot 3D Arrows (Quiver)
            ax.quiver(
                x_coords * 1e9,
                y_coords * 1e9,
                z_coords * 1e9,
                u_comp,
                v_comp,
                w_comp,
                length=4.0,
                normalize=True,
                color='royalblue',
                alpha=0.8
            )

            # Styling
            ax.set_title(f"MuMax3 Simulation - Frame {frame_idx}")
            ax.set_xlabel('x (nm)')
            ax.set_ylabel('y (nm)')
            ax.set_zlabel('z (nm)')

            ax.set_xlim(pmin[0] * 1e9, pmax[0] * 1e9)
            ax.set_ylim(pmin[1] * 1e9, pmax[1] * 1e9)
            ax.set_zlim(pmin[2] * 1e9, pmax[2] * 1e9)

            return ax,

        # 4. Create and Display Animation
        frames_to_render = len(data_frames)
        # Cap frames if too many to prevent browser crash, though 50 is fine
        if frames_to_render > 60:
            frames_to_render = 60
            print(f"Limiting video to first {frames_to_render} frames.")

        ani = animation.FuncAnimation(
            fig,
            update_frame,
            frames=frames_to_render,
            interval=150,
            blit=False
        )

        plt.close()
        display(HTML(ani.to_jshtml()))
    except Exception as err:
        print(f"\nERROR: Error during visualization: {err}")

In [ ]:
"""
Download Results (Individual Files).

Automatically save the plot and video and download them individually.
NOTE: You may need to 'Allow multiple downloads' in your browser if prompted.
"""

import subprocess
import time

import matplotlib.pyplot as plt
from google.colab import files

print("=== 6. Saving and Downloading Files ===")

# Ensure FFmpeg is installed for video saving
if subprocess.call("which ffmpeg", shell=True) != 0:
    print("Installing FFmpeg...")
    subprocess.run("apt install -qq ffmpeg", shell=True, check=False)

# --- 1. Save and Download Plot ---
try:
    print("Generating 'signals_plot.png'...")
    # Recalculate signals
    t, x_t = generate_xt(
        omega_lo_slider.value,
        i_amp_slider.value,
        q_amp_slider.value,
        t_max_slider.value,
        dt_slider.value
    )
    y_t = apply_nonlinearity(
        x_t,
        alpha1_slider.value,
        alpha2_slider.value,
        alpha3_slider.value
    )
    a_1db, a_ip3 = compute_metrics(alpha1_slider.value, alpha3_slider.value)

    # Reuse the plotting function from Step 2
    fig = create_signal_figure(t, x_t, y_t, a_1db, a_ip3)

    filename_plot = 'signals_plot.png'
    plt.savefig(filename_plot)
    plt.close(fig)
    print(f"Saved {filename_plot}")

    print(f"Downloading {filename_plot}...")
    files.download(filename_plot)
except Exception as e:
    print(f"ERROR: Error saving plot: {e}")

# Short delay to prevent browser from blocking the second download immediately
time.sleep(2)

# --- 2. Save and Download Video ---
try:
    filename_video = 'simulation_video.mp4'
    if 'ani' in globals():
        print(
            f"Saving '{filename_video}' from Step 5 animation... "
            "(This may take a moment)"
        )
        ani.save(filename_video, fps=15)
        print(f"Saved {filename_video}")

        print(f"Downloading {filename_video}...")
        files.download(filename_video)
    else:
        print(
            "WARNING: Animation object 'ani' not found. "
            "Please run Step 5 first."
        )
except Exception as e:
    print(f"ERROR: Error saving video: {e}")